# Wallet Risk Scorer
This notebook calculates risk scores (0-1000) for wallets interacting with Compound V2 based on their lending/borrowing history.

In [ ]:
!pip install requests pandas


In [20]:
from google.colab import files
import pandas as pd
import io

# Upload the CSV file from your local machine
uploaded = files.upload()

# Read the uploaded CSV into a DataFrame
filename = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[filename]))

# Extract wallet addresses in lowercase
wallets = df['wallet_id'].str.lower().tolist()

# Preview the first 5 wallets
wallets[:5]


Saving Wallet id - Sheet1.csv to Wallet id - Sheet1 (3).csv


['0x0039f22efb07a647557c7c5d17854cfd6d489ef3',
 '0x06b51c6882b27cb05e712185531c1f74996dd988',
 '0x0795732aacc448030ef374374eaae57d2965c16c',
 '0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9',
 '0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae']

In [23]:
import requests
from time import sleep
from tqdm.notebook import tqdm

COVALENT_API_KEY = "cqt_rQcxxP6mgqFfD8gfyyDKHKHrcBch"
COVALENT_BASE_URL = "https://api.covalenthq.com/v1"

def fetch_compound_v2_transactions(wallet):
    url = f"{COVALENT_BASE_URL}/1/address/{wallet}/transactions_v2/"
    params = {
        "key": COVALENT_API_KEY,
        "page-size": 10000
    }
    try:
        res = requests.get(url, params=params)
        data = res.json()
        return data.get("data", {}).get("items", [])
    except Exception as e:
        print(f"⚠️ Error fetching tx for {wallet}: {e}")
        return []

wallet_tx_map = {}

# 🔄 Now using all 103 wallets
for i, wallet in enumerate(tqdm(wallets), 1):
    print(f"🔎 Fetching tx for {wallet} ({i}/{len(wallets)})")
    txs = fetch_compound_v2_transactions(wallet)
    wallet_tx_map[wallet] = txs
    sleep(1.5)  # ⏱️ Delay to avoid hitting API rate limits


  0%|          | 0/103 [00:00<?, ?it/s]

🔎 Fetching tx for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3 (1/103)
🔎 Fetching tx for 0x06b51c6882b27cb05e712185531c1f74996dd988 (2/103)
🔎 Fetching tx for 0x0795732aacc448030ef374374eaae57d2965c16c (3/103)
🔎 Fetching tx for 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9 (4/103)
🔎 Fetching tx for 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae (5/103)
🔎 Fetching tx for 0x104ae61d8d487ad689969a17807ddc338b445416 (6/103)
🔎 Fetching tx for 0x111c7208a7e2af345d36b6d4aace8740d61a3078 (7/103)
🔎 Fetching tx for 0x124853fecb522c57d9bd5c21231058696ca6d596 (8/103)
🔎 Fetching tx for 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc (9/103)
🔎 Fetching tx for 0x1656f1886c5ab634ac19568cd571bc72f385fdf7 (10/103)
🔎 Fetching tx for 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22 (11/103)
🔎 Fetching tx for 0x19df3e87f73c4aaf4809295561465b993e102668 (12/103)
🔎 Fetching tx for 0x1ab2ccad4fc97c9968ea87d4435326715be32872 (13/103)
🔎 Fetching tx for 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f (14/103)
🔎 Fetching tx for 0x1e43dacdc

This block uses the Covalent API to fetch transaction history for each wallet. It loops through each address and queries for all transactions, storing them in a dictionary for later processing. The delay helps avoid API rate-limiting errors.

In [24]:
# ✅ Compound V2 contract addresses
COMPOUND_V2_ADDRESSES = {
    "0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b",  # Comptroller
    "0x39aa39c021dfbae8fac545936693ac917d5e7563",  # cUSDC
    "0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643",  # cDAI
    "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",  # cETH
    "0x6c8c6b02e7b2be14d4fa6022dfd6d2bba7e3b0b9",  # cBAT
    "0xf5dce57282a584d2746faf1593d3121fcac444dc",  # cREP
    "0xf650c3d88d12db855b8bf7d11be6c55a4e07ddb9",  # cUSDT
    "0x95b4ef2869ebd94beb4eee400a99824bf5dc325b",  # cCOMP
}

def filter_compound_tx(txs):
    return [
        tx for tx in txs
        if tx and tx.get("to_address") and tx.get("to_address", "").lower() in COMPOUND_V2_ADDRESSES
    ]

# 🧪 Filtered dictionary: only Compound V2 transactions per wallet
compound_tx_map = {}

for wallet, txs in wallet_tx_map.items():
    compound_txs = filter_compound_tx(txs)
    compound_tx_map[wallet] = compound_txs


This block filters each wallet's transaction history to retain only those involving known Compound V2 contract addresses. It helps isolate relevant DeFi activity specific to Compound.

In [25]:
import datetime
from dateutil import parser as date_parser

def extract_features(compound_tx_map):
    features = []
    now = datetime.datetime.now(datetime.timezone.utc)

    for wallet, txs in compound_tx_map.items():
        tx_count = len(txs)

        unique_contracts = set()
        max_tx_value = 0
        total_gas = 0
        total_gas_price = 0
        last_activity = None

        for tx in txs:
            to_addr = tx.get("to_address")
            if to_addr:
                unique_contracts.add(to_addr.lower())

            value_eth = int(tx.get("value", 0)) / 1e18
            max_tx_value = max(max_tx_value, value_eth)

            gas_price = int(tx.get("gas_price", 0))
            gas_used = int(tx.get("gas_used", 0))
            total_gas += gas_used
            total_gas_price += gas_price * gas_used

            tx_time = tx.get("block_signed_at")
            if tx_time:
                tx_datetime = date_parser.parse(tx_time)
                if last_activity is None or tx_datetime > last_activity:
                    last_activity = tx_datetime

        avg_gas_fee_eth = (total_gas_price / total_gas / 1e18) if total_gas > 0 else 0
        days_since_activity = (now - last_activity).days if last_activity else 9999

        features.append({
            "wallet_id": wallet,
            "compound_tx_count": tx_count,
            "unique_contracts_interacted": len(unique_contracts),
            "max_tx_value_eth": round(max_tx_value, 6),
            "avg_gas_fee_eth": round(avg_gas_fee_eth, 6),
            "last_activity_days_ago": days_since_activity
        })

    return pd.DataFrame(features)

# 🛠 Extract features for scoring
features_df = extract_features(compound_tx_map)
features_df.head()


,wallet_id,compound_tx_count,unique_contracts_interacted,max_tx_value_eth,avg_gas_fee_eth,last_activity_days_ago
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,18,3,100.0,0,1319
1,0x06b51c6882b27cb05e712185531c1f74996dd988,1,1,0.0,0,1745
2,0x0795732aacc448030ef374374eaae57d2965c16c,1,1,0.0,0,1704
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,1,1,0.0,0,1688
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,1,1,0.0,0,1739


This part extracts numerical features from the filtered Compound V2 transactions:

Total number of transactions

Number of unique contracts interacted with

Max ETH transferred

Average gas fee spent

Days since last activity
These features are critical for evaluating wallet risk.

In [26]:
from sklearn.preprocessing import MinMaxScaler

def compute_risk_scores(features_df):
    scoring_df = features_df.copy()

    # Reverse the 'last_activity_days_ago' so recent activity gets higher score
    scoring_df['recency'] = scoring_df['last_activity_days_ago'].max() - scoring_df['last_activity_days_ago']

    # Select features to normalize
    score_features = [
        'compound_tx_count',
        'unique_contracts_interacted',
        'max_tx_value_eth',
        'avg_gas_fee_eth',
        'recency'
    ]

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(scoring_df[score_features])

    # Simple weighted sum: give equal weight (can be tuned later)
    raw_scores = scaled.sum(axis=1)

    # Normalize to 0–1000
    normalized_scores = MinMaxScaler((0, 1000)).fit_transform(raw_scores.reshape(-1, 1)).flatten()

    scoring_df['score'] = normalized_scores.round().astype(int)

    return scoring_df[['wallet_id', 'score']]

# 📊 Get final risk scores
risk_scores_df = compute_risk_scores(features_df)
risk_scores_df.head()


,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,1000
1,0x06b51c6882b27cb05e712185531c1f74996dd988,363
2,0x0795732aacc448030ef374374eaae57d2965c16c,365
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,365
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,363


This block uses min-max scaling to normalize feature values between 0 and 1. It inverts last_activity_days_ago so that recent activity gets higher scores. Then, a weighted sum of all normalized features is calculated to produce a final risk score from 0 to 1000 for each wallet.

In [27]:
# 💾 Save the risk scores to a CSV file
risk_scores_df.to_csv("wallet_risk_scores.csv", index=False)
print("✅ Saved: wallet_risk_scores.csv")


✅ Saved: wallet_risk_scores.csv
